In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
team_urls = pd.read_csv(os.getcwd() + '/data/cap_space_all_teams.csv')
team_codes = pd.read_csv(os.getcwd() + '/data/team_codes.csv')

# combine all tables that aren't 'cap_totals' into one table for each team and year
for team_name in team_codes.keys():
    team_code = team_codes[team_name]
    team_path = os.getcwd() + '/data/teams/' + team_name
    years = [year for year in os.listdir(team_path) if os.path.isdir(team_path + '/' + year)]
    for year in years:
        files = os.listdir(team_path + '/' + year)
        files = [file for file in files if file.endswith('_cleaned.csv') and 'cap_totals' not in file]
        # print(f'{team_name}: {files}')
        all_players = pd.DataFrame()
        for file in files:
            df = pd.read_csv(team_path + '/' + year + '/' + file)
            # check that none of df columns contain 'Cap Maximum Summary'
            if df.columns.str.contains('Cap Maximum Summary').any():
                print(f'{file} contains Cap Maximum Summary')
                df = df.loc[:, ~df.str.contains('Cap Maximum Summary')]
                df = df.dropna(axis=1, how='all')
            all_players = pd.concat([all_players, df])
        all_players.to_csv(team_path + '/' + year + '/' + team_code + '_' + year + '_all_players_cleaned.csv', index=False)

In [ ]:
# combine every all_players table into one table for each team with column for year
for team_name in team_codes.keys():
    team_code = team_codes[team_name]
    team_path = os.getcwd() + '/data/teams/' + team_name
    years = [year for year in os.listdir(team_path) if os.path.isdir(team_path + '/' + year)]
    all_players = pd.DataFrame()
    for year in years:
        df = pd.read_csv(team_path + '/' + year + '/' + team_code + '_' + year + '_all_players_cleaned.csv')
        df['team_name'] = team_name
        df['year'] = year
        df = df[['team_name', 'year'] + [col for col in df.columns if col not in ['team_name', 'year']]]
        all_players = pd.concat([all_players, df])
    all_players.to_csv(team_path + '/' + team_code + '_all_players_cleaned.csv', index=False)

In [ ]:
# combine all of all_players_cleaned tables into one table
all_players = pd.DataFrame()
for team_name in team_codes.keys():
    team_code = team_codes[team_name]
    team_path = os.getcwd() + '/data/teams/' + team_name
    files = os.listdir(team_path)
    files = [file for file in files if file.endswith('_all_players_cleaned.csv')]
    if len(files) == 0:
        print(f'{team_name} has no all_players_cleaned.csv')
        continue
    elif len(files) > 1:
        print(f'{team_name} has more than one all_players_cleaned.csv')
        continue
    df = pd.read_csv(team_path + '/' + files[0])
    all_players = pd.concat([all_players, df])

all_players.to_csv(os.getcwd() + '/data/' + 'all_players_cleaned.csv', index=False)